In [2]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np

from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_csv('AI_Human.csv')
texts = df['text']
labels = df['generated']
text_train, text_val, label_train, label_val = train_test_split(texts, labels, test_size=.1, random_state=12)
text_train, text_test, label_train, label_test = train_test_split(text_train, label_train, test_size=.2, random_state=12)
print('train:' + str(text_train.shape))
print('test:' + str(text_test.shape))
print('val:' + str(text_val.shape))

train:(350808,)
test:(87703,)
val:(48724,)


In [4]:
lengths = text_train.str.split().apply(len)

print("Min length:", np.min(lengths))
print("Max length:", np.max(lengths))
print("Mean length:", np.mean(lengths))

percentile_95 = int(np.percentile(lengths, 95))
print("95th percentile length:", percentile_95)

# detirmines the max len of text to vectorize
max_len = percentile_95 # A good choice is the 95th percentile for len


Min length: 0
Max length: 1668
Mean length: 393.14396193929446
95th percentile length: 722


In [ ]:
max_tokens = 5000   # vocabulary size, counts tokens/words by frequency and any token not in the top 5000 all become

vectorizer = layers.TextVectorization(
    max_tokens=max_tokens,
    output_mode="int",
    output_sequence_length=max_len
)

# Adapt vectorizer on your text data
vectorizer.adapt(text_train)
vectorizer.adapt(text_tes)
# Vectorize dataset
X = vectorizer(np.array(texts))
X = vectorizer(np.array(texts))

y = np.array(labels)

In [6]:
embed_dim = 64
hidden_dim = 128

model = models.Sequential([
    vectorizer,  # tokenization built into the model
    layers.Embedding(max_tokens, embed_dim, mask_zero=True),
    layers.GlobalAveragePooling1D(),
    layers.Dense(hidden_dim, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
history = model.fit(
    train_X, train_y,
    validation_data=(val_X, val_y),
    epochs=5,
    batch_size=2
)
